## Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Filter the DataFrame using input statements for minimum and maximum temperature preferences.
2. Drop any empty rows on the new DataFrame. 
3. Find the nearest hotel to each city using each city’s latitude and longitude and the Google Nearby Search.  
4. Create a new DataFrame where the rows that do not have a hotel are dropped from the DataFrame created in Step 4. 
5. Add the cities to a marker layer map with a pop-up marker for each city that includes: 
    - Hotel name
    - City
    - Country
    - Current weather description and the maximum temperature
6. Export the DataFrame as a CSV file into the Vacation_Search folder as WeatherPy_vacation.csv.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Marabba,SD,12.3500,32.1833,78.04,69,83,13.42,broken clouds
1,1,Tiksi,RU,71.6872,128.8694,45.82,76,80,7.58,broken clouds
2,2,Lebu,CL,-37.6167,-73.6500,52.45,96,100,12.12,overcast clouds
3,3,Touros,BR,-5.1989,-35.4608,71.65,93,100,9.28,overcast clouds
4,4,Tuktoyaktuk,CA,69.4541,-133.0374,59.00,82,75,8.05,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria.. 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 20
What is the maximum temperature you would like for your vacation? 40


In [4]:
# 3. Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)] 
                                    
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Ushuaia,AR,-54.8000,-68.3000,31.66,93,75,26.46,snow
66,66,Pevek,RU,69.7008,170.3133,31.87,100,100,8.55,light snow
74,74,Tasiilaq,GL,65.6145,-37.6368,33.96,83,84,1.86,broken clouds
92,92,Barrow,US,71.2906,-156.7887,39.22,87,100,18.41,mist
96,96,Leningradskiy,RU,69.3833,178.4167,34.14,97,100,3.96,overcast clouds
130,130,Punta Arenas,CL,-53.1500,-70.9167,33.82,86,20,17.27,few clouds
186,186,Iqaluit,CA,63.7506,-68.5145,38.93,100,20,2.30,few clouds
200,200,Clyde River,CA,70.4692,-68.5914,37.63,100,100,5.75,light rain
244,244,Ancud,CL,-41.8697,-73.8203,39.22,89,100,5.50,overcast clouds
351,351,San Carlos De Bariloche,AR,-41.1456,-71.3082,33.24,100,100,3.44,heavy snow


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                13
City                   13
Country                13
Lat                    13
Lng                    13
Max Temp               13
Humidity               13
Cloudiness             13
Wind Speed             13
Current Description    13
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Ushuaia,AR,-54.8000,-68.3000,31.66,93,75,26.46,snow
66,66,Pevek,RU,69.7008,170.3133,31.87,100,100,8.55,light snow
74,74,Tasiilaq,GL,65.6145,-37.6368,33.96,83,84,1.86,broken clouds
92,92,Barrow,US,71.2906,-156.7887,39.22,87,100,18.41,mist
96,96,Leningradskiy,RU,69.3833,178.4167,34.14,97,100,3.96,overcast clouds


In [7]:
clean_travel_cities.count()

City_ID                13
City                   13
Country                13
Lat                    13
Lng                    13
Max Temp               13
Humidity               13
Cloudiness             13
Wind Speed             13
Current Description    13
dtype: int64

In [8]:
clean_travel_cities.columns

Index(['City_ID', 'City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Current Description'],
      dtype='object')

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Ushuaia,AR,31.66,snow,-54.8000,-68.3000,
66,Pevek,RU,31.87,light snow,69.7008,170.3133,
74,Tasiilaq,GL,33.96,broken clouds,65.6145,-37.6368,
92,Barrow,US,39.22,mist,71.2906,-156.7887,
96,Leningradskiy,RU,34.14,overcast clouds,69.3833,178.4167,
130,Punta Arenas,CL,33.82,few clouds,-53.1500,-70.9167,
186,Iqaluit,CA,38.93,few clouds,63.7506,-68.5145,
200,Clyde River,CA,37.63,light rain,70.4692,-68.5914,
244,Ancud,CL,39.22,overcast clouds,-41.8697,-73.8203,
351,San Carlos De Bariloche,AR,33.24,heavy snow,-41.1456,-71.3082,


In [10]:
# 6a. Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Ushuaia,AR,31.66,snow,-54.8000,-68.3000,Albatross Hotel
66,Pevek,RU,31.87,light snow,69.7008,170.3133,Severnoye Zoloto
74,Tasiilaq,GL,33.96,broken clouds,65.6145,-37.6368,The Red House
92,Barrow,US,39.22,mist,71.2906,-156.7887,King Eider Inn
96,Leningradskiy,RU,34.14,overcast clouds,69.3833,178.4167,Gostinitsa
130,Punta Arenas,CL,33.82,few clouds,-53.1500,-70.9167,Hotel Hain
186,Iqaluit,CA,38.93,few clouds,63.7506,-68.5145,Frobisher Inn
200,Clyde River,CA,37.63,light rain,70.4692,-68.5914,Qamaq Hotel
244,Ancud,CL,39.22,overcast clouds,-41.8697,-73.8203,Hotel Arena Gruesa
351,San Carlos De Bariloche,AR,33.24,heavy snow,-41.1456,-71.3082,Hotel Patagonia


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Ushuaia,AR,31.66,snow,-54.8000,-68.3000,Albatross Hotel
66,Pevek,RU,31.87,light snow,69.7008,170.3133,Severnoye Zoloto
74,Tasiilaq,GL,33.96,broken clouds,65.6145,-37.6368,The Red House
92,Barrow,US,39.22,mist,71.2906,-156.7887,King Eider Inn
96,Leningradskiy,RU,34.14,overcast clouds,69.3833,178.4167,Gostinitsa
130,Punta Arenas,CL,33.82,few clouds,-53.1500,-70.9167,Hotel Hain
186,Iqaluit,CA,38.93,few clouds,63.7506,-68.5145,Frobisher Inn
200,Clyde River,CA,37.63,light rain,70.4692,-68.5914,Qamaq Hotel
244,Ancud,CL,39.22,overcast clouds,-41.8697,-73.8203,Hotel Arena Gruesa
351,San Carlos De Bariloche,AR,33.24,heavy snow,-41.1456,-71.3082,Hotel Patagonia


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

###  Add a pop-up markers to each city. 
---
#### In the pop-up for each city add: 
1. The city name
2. The country code
3. The weather description and maximum temperature for the city

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))